In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [2]:
### include useful folders
import sys

sys.path.append("../vendors/mtl_girnet/data_prep/")

import json
import h5py
import numpy as np
import glob
import random
import pandas as pd
import re
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib import pyplot as plt

# nltk
import nltk

# tokenizer
from twokenize import tokenizeRawTweetText as tokenize

# for a particular dataset
from xml.dom import minidom

In [3]:

### SemEval 2017 Task A

df = pd.read_csv("../data/datastories-semeval2017-task4/dataset/Subtask_A/4A-English/SemEval2017-task4-dev.subtask-A.english.INPUT.txt", sep="\t", header=None)

decode_map = {"negative": -1, "neutral": 0, "positive": 1}

df[1] = df[1].apply(lambda x: decode_map[x])
df[2] = df[2].apply(lambda x: tokenize(x))

data = map( lambda x :{'sentiment': x[1] , 'tokens': x[2] , 'text': ' '.join(x[2])} , df.to_numpy() )

en_semeval_17 = list(data)


### English-Spanish Code Mixed Data 

sents = {"N":-1 , "P" :1 , "NONE":0}

data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/cs-corpus-with-tweets_train.txt", encoding='utf-8').read().split("\n") 
data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': sents[x[1]] , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )
en_es_wssa_data_train = list(data)

data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/cs-corpus-with-tweets_test.txt", encoding='utf-8').read().split("\n") 
data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': sents[x[1]] , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )
en_es_wssa_data_test = list(data)

en_es_wssa_data = list(en_es_wssa_data_train) + list(en_es_wssa_data_test)

### Spanish Tweet Dataset

xmldoc = minidom.parse("../vendors/mtl_girnet/data_prep/data_cm_senti/general-tweets-train-tagged.xml")
tweets = xmldoc.getElementsByTagName('tweet')

sents = {"N":-1 , "P" :1 , "NEU":0 , 'NONE':0 , "P+" : 1 , "N+":-1 }


es_tass1_data = []

for i in range( len(tweets)-1) :
    if i == 6055:
        continue # bad jogar
    textt = tweets[i].getElementsByTagName('content')[0].childNodes[0].data
    words = tokenize( textt )
    sentiment = tweets[i].getElementsByTagName('polarity')[0].getElementsByTagName('value')[0].childNodes[0].data
    assert len(tweets[i].getElementsByTagName('polarity')[0].getElementsByTagName('entity'))==0
    es_tass1_data.append({'text':textt , 'tokens':words , 'sentiment': sents[sentiment] })

### Some english tweet data

data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/twitter4242.txt", "r", encoding="utf-8",errors='ignore').read().split("\n")[1:-1]
data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': int(np.sign(int(x[0])-int(x[1]))) , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )

en_twitter_data = list(data)

### es2_twitter_data

data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/1600_tweets_dev_complete.txt", encoding="utf-8").read().split("\n")[1:-1]
data += open("../vendors/mtl_girnet/data_prep/data_cm_senti/1600_tweets_test_average_complete.tsv", encoding="utf-8").read().split("\n")[1:-2]

data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': int(np.sign(int(x[0])-int(x[1]))) , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )

es2_twitter_data = list(data)

def get_y(data):
    from keras.utils import to_categorical
    y = []
    for row in data:
        y.append(int(row['sentiment']))
    y = to_categorical(y,num_classes=3)
    return y


print("Code-Mixed: en_es_wssa_data: %d" % len(en_es_wssa_data))
print("Spanish: es2_twitter_data: %d" % len(es2_twitter_data))
print("Spanish: es_tass1_data: %d" % len(es_tass1_data))
print("English: en_twitter_data: %d" % len(en_twitter_data))
# print("English: en_sentiment140: %d" %len(en_sentiment140))
en_es_y =  get_y(en_es_wssa_data)
en_es_y_train =  get_y(en_es_wssa_data_train)
en_es_y_test =  get_y(en_es_wssa_data_test)
es_twitter_y = get_y(es2_twitter_data)
es_tass_y = get_y(es_tass1_data)
en_twitter_y = get_y(en_twitter_data)
en_semeval_17_y = get_y(en_semeval_17)
# en_sentiment140_y = get_y(en_sentiment140)

Code-Mixed: en_es_wssa_data: 3062
Spanish: es2_twitter_data: 3202
Spanish: es_tass1_data: 7217
English: en_twitter_data: 4241


Using TensorFlow backend.


In [4]:
# len(en_semeval_17)

In [5]:
# sentiment_analysis = []

In [6]:
# data = en_es_wssa_data_train
# lang = "cm"
# for sent in data:
#     sentiment_analysis.append("\t".join([sent['text'], str(sent['sentiment']), lang, "\n"]))

In [7]:
# with open("train.txt", "w") as f:
#     f.writelines(sentiment_analysis)

In [8]:
from tensorflow.keras import backend as K


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
def get_class_weight(y):
    """
    Used from: https://stackoverflow.com/a/50695814
    TODO: check validity and 'balanced' option
    :param y: A list of one-hot-encoding labels [[0,0,1,0],[0,0,0,1],..]
    :return: class-weights to be used by keras model.fit(.. class_weight="") -> {0:0.52134, 1:1.adas..}
    """
    y_integers = np.argmax(y, axis=1)
    class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
    d_class_weights = dict(enumerate(class_weights))
    return d_class_weights

from keras import backend as K
from keras import losses

def loss_ordinal(y_true, y_pred):
    weights = K.cast(K.abs(K.argmax(y_true, axis=1) - K.argmax(y_pred, axis=1))/(K.int_shape(y_pred)[1] - 1), dtype='float32')
    return (1.0 + weights) * losses.categorical_crossentropy(y_true, y_pred)

In [9]:
# ! pip install bpemb

In [10]:
from bpemb import BPEmb
multibpemb = BPEmb(lang="multi", vs=1000000, dim=200)

Setting dim=300 for multilingual BPEmb


paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [11]:
weight_matrix = multibpemb.vectors

In [12]:
multibpemb

BPEmb(lang=multi, vs=1000000, dim=300)

In [13]:
max_len = 32
zero_vector = [0 for _ in range(300)]
def get_x(data_):
    x_  = []
    for sent in data_:
        pred = list(multibpemb.embed(sent['text']))
        if len(pred) >= 32:
            pred = pred[:32]
        else:
            counter = len(pred)
            while counter < max_len:
                pred.append(zero_vector)
                counter = counter + 1
        x_.append(pred)
    return np.array(x_)
en_es_x =  get_x(en_es_wssa_data)
es_twitter_x = get_x(es2_twitter_data)
es_tass_x = get_x(es_tass1_data)
en_twitter_x = get_x(en_twitter_data)
en_semeval_17_x = get_x(en_semeval_17)
en_es_x_train =  get_x(en_es_wssa_data_train)
en_es_x_test =  get_x(en_es_wssa_data_test)

In [199]:
# GET Y PADDED TOKENS NUMBER
max_len = 32
def get_x(data_):
    x_  = []
    for sent in data_:
        pred = list(multibpemb.encode_ids(sent['text']))
        if len(pred) >= 32:
            pred = pred[:32]
        else:
            counter = len(pred)
            while counter < max_len:
                pred.append(0)
                counter = counter + 1
        x_.append(pred)
    return np.array(x_)
en_es_x =  get_x(en_es_wssa_data)
es_twitter_x = get_x(es2_twitter_data)
es_tass_x = get_x(es_tass1_data)
en_twitter_x = get_x(en_twitter_data)
en_semeval_17_x = get_x(en_semeval_17)
en_es_x_train =  get_x(en_es_wssa_data_train)
en_es_x_test =  get_x(en_es_wssa_data_test)

In [32]:
import fasttext

In [33]:
embed = fasttext.load_model('../vendors/language-models/all_p_fasttext.bin')

In [34]:
max_len = 32
zero_vector = [0 for _ in range(100)]
def get_x(data_):
#     x_  = []
#     for sent in data_:
#         x_.append(embed.get_sentence_vector(sent['text'].replace("\n"," ")))
#     return np.array(x_)
    x_  = []
    for sent in data_:
        tokenised = fasttext.tokenize(sent['text'])
        sent_vector = []
        counter = 0
        for token in tokenised:
            if counter >= max_len:
                break
            else:
                sent_vector.append(embed[token])
                counter = counter + 1
        
        if counter < max_len:
            sent_vector.append(embed['</s>'])
            counter = counter + 1
                               
        while counter < max_len:
            sent_vector.append(zero_vector)
            counter = counter + 1
            
        x_.append(sent_vector)
        
    return np.array(x_)

en_es_x =  get_x(en_es_wssa_data)
es_twitter_x = get_x(es2_twitter_data)
es_tass_x = get_x(es_tass1_data)
en_twitter_x = get_x(en_twitter_data)
en_semeval_17_x = get_x(en_semeval_17)
en_es_x_train =  get_x(en_es_wssa_data_train)
en_es_x_test =  get_x(en_es_wssa_data_test)

In [15]:
from keras.layers import *
from keras.models import Sequential
from keras.preprocessing import sequence
from attention_lstm import AttentionWithContext

In [ ]:
model = Sequential()
model.add(Dense(50, input_shape=(100,)))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy', f1])

In [16]:
model = Sequential()
model.add(Bidirectional(LSTM(50, dropout=0.3, input_shape=(32, 300), recurrent_dropout=0.3, return_sequences=True)))
model.add(AttentionWithContext())
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss=loss_ordinal,
              optimizer='adam',
              metrics=['accuracy', f1])

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
# !pip install keras-self-attention
from keras_self_attention import SeqSelfAttention

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(50, input_shape=(None, 32, 100))))
# model.add(SeqSelfAttention(attention_activation='sigmoid'))
# model.add(Flatten())
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss=loss_ordinal,
              optimizer='adam',
              metrics=['accuracy', f1])

In [ ]:
# Convolution
kernel_size = 5
filters = 64
pool_size = 4

model = Sequential()
model.add(Conv1D(3,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1, input_shape=(32, 300)))
# model.add(MaxPooling1D(pool_size=pool_size))
# model.add(Conv1D(5,
#                  64,
#                  padding='valid',
#                  activation='relu',
#                  strides=1))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(50, dropout=0.3, recurrent_dropout=0.3)))
model.add(Dropout(0.3))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', f1])

In [20]:
es = EarlyStopping(monitor='val_f1', mode='max', verbose=1, patience=5, restore_best_weights=True)

In [ ]:
# history = model.fit(en_es_x_train, en_es_y_train, epochs=10, shuffle=True, validation_data=(en_es_x_test
#                                                                                             ,en_es_y_test))

In [ ]:
# model.evaluate(X_test, y_test)

In [21]:
history = model.fit(en_semeval_17_x, en_semeval_17_y, epochs=20, initial_epoch=0, validation_split=0.2, shuffle=True, callbacks=[es])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 16505 samples, validate on 4127 samples
Epoch 1/20
16505/16505 [==============================] - 207s 13ms/step - loss: 1.2235 - acc: 0.5872 - f1: 0.4265 - val_loss: 1.3033 - val_acc: 0.5658 - val_f1: 0.5206
Epoch 2/20
16505/16505 [==============================] - 59s 4ms/step - loss: 1.0800 - acc: 0.6474 - f1: 0.5909 - val_loss: 1.2161 - val_acc: 0.5958 - val_f1: 0.5491
Epoch 3/20
16505/16505 [==============================] - 58s 4ms/step - loss: 1.0156 - acc: 0.6741 - f1: 0.6304 - val_loss: 1.2169 - val_acc: 0.6089 - val_f1: 0.5821
Epoch 4/20
16505/16505 [==============================] - 59s 4ms/step - loss: 0.9712 - acc: 0.6899 - f1: 0.6574 - val_loss: 1.1803 - val_acc: 0.6128 - val_f1: 0.5742
Epoch 5/20
16505/16505 [==============================] - 59s 4ms/step - loss: 0.9356 - acc: 0.7005 - f1: 0.6738 - val

In [22]:
model.evaluate(en_es_x_test, en_es_y_test)

613/613 [==============================] - 3s 5ms/step


[2.5036796031923902, 0.5236541600639629, 0.5194763644290009]

In [ ]:
history = model.fit(es_tass_x, es_tass_y, epochs=35, initial_epoch=20, validation_split=0.2, shuffle=True, callbacks=[es])

Train on 5773 samples, validate on 1444 samples
Epoch 21/35
5773/5773 [==============================] - 99s 17ms/step - loss: 1.5766 - acc: 0.4731 - f1: 0.3279 - val_loss: 1.2728 - val_acc: 0.5845 - val_f1: 0.4095
Epoch 22/35
5773/5773 [==============================] - 99s 17ms/step - loss: 1.2965 - acc: 0.5597 - f1: 0.4235 - val_loss: 1.2187 - val_acc: 0.6066 - val_f1: 0.4645
Epoch 23/35
5773/5773 [==============================] - 100s 17ms/step - loss: 1.2240 - acc: 0.5948 - f1: 0.4916 - val_loss: 1.1890 - val_acc: 0.6115 - val_f1: 0.4895
Epoch 24/35
5773/5773 [==============================] - 100s 17ms/step - loss: 1.1629 - acc: 0.6248 - f1: 0.5262 - val_loss: 1.1804 - val_acc: 0.6157 - val_f1: 0.5274
Epoch 25/35
5773/5773 [==============================] - 100s 17ms/step - loss: 1.1160 - acc: 0.6409 - f1: 0.5725 - val_loss: 1.1644 - val_acc: 0.6288 - val_f1: 0.5711
Epoch 26/35
5773/5773 [==============================] - 101s 18ms/step - loss: 1.0734 - acc: 0.6584 - f1: 0.5998 

In [ ]:
model.evaluate(en_es_x_test, en_es_y_test)

In [ ]:
history = model.fit(es_twitter_x, es_twitter_y, epochs=40, initial_epoch=35, validation_split=0.2, shuffle=True,  callbacks=[es])

In [ ]:
model.evaluate(en_es_x_test, en_es_y_test)

In [ ]:
x = np.concatenate([en_semeval_17_x, en_twitter_x, es_tass_x, es_twitter_x])
y = np.concatenate([en_semeval_17_y, en_twitter_y, es_tass_y, es_twitter_y])

In [ ]:
history = model.fit(x, y, epochs=60, initial_epoch=40,validation_data=(en_es_x_train,en_es_y_train), shuffle=True, callbacks=[es])

In [ ]:
model.evaluate(en_es_x_test, en_es_y_test)

In [ ]:
history = model.fit(en_es_x_train, en_es_y_train, epochs=80, initial_epoch=60, validation_data=(en_es_x_test, en_es_y_test), shuffle=True,  callbacks=[es])

In [ ]:
model.evaluate(en_es_x_test, en_es_y_test)

In [ ]:
import tensorflow as tf
tf.reset_default_graph()

# MTL 

In [18]:
import tensorflow.keras as keras
from keras.layers import *
from keras.models import Model, Sequential
from attention_lstm import AttentionWithContext
from keras.callbacks import *
# from keras_self_attention import SeqSelfAttention

In [262]:
class GiretTwoCell(keras.layers.Layer):

    def __init__(self, cell_1 , cell_2 , nHidden , **kwargs):
        self.cell_1 = cell_1
        self.cell_2 = cell_2
        self.nHidden = nHidden
        self.state_size = [nHidden,nHidden]
        super(GiretTwoCell, self).__init__(**kwargs)

    def build(self, input_shape):
        
        nHidden = self.nHidden
        
        input_shape_n = ( input_shape[0] , input_shape[1]- 2 )
#         print "pp", input_shape_n
        
#         self.cell_1.build(input_shape_n)
#         self.cell_2.build(input_shape_n)
        
        self._trainable_weights += ( self.cell_1.trainable_weights )
        self._trainable_weights += ( self.cell_2.trainable_weights )
        
        self._non_trainable_weights += (  self.cell_1.non_trainable_weights )
        self._non_trainable_weights += (  self.cell_2.non_trainable_weights )
        
        self.built = True

    def call(self, inputs, states):
        
        nHidden = self.nHidden
        
        gate_val_1 = inputs[ : , 0:1]
        gate_val_2 = inputs[ : , 1:2]
        
        inputs  = inputs[ : , 2: ]
                
        gate_val_1 = K.repeat_elements(gate_val_1 , nHidden , -1 ) # shape # bs , hidden
        gate_val_2 = K.repeat_elements(gate_val_2 , nHidden , -1 ) # shape # bs , hidden
        
        _ , [h1 , c1 ]  = self.cell_1.call( inputs , states )
        _ , [h2 , c2 ]  = self.cell_2.call( inputs , states )
        
        h = gate_val_1*h1 + gate_val_2*h2  + (1 - gate_val_1 -  gate_val_2 )*states[0]
        c = gate_val_1*c1 + gate_val_2*c2  + (1 - gate_val_1 -  gate_val_2 )*states[1]
        
        return h, [h , c ]

In [279]:
hidden = 100
dims = 300

rnn_en = LSTM(hidden, name='en_lstm', recurrent_dropout=0.3, dropout=0.3)
rnn_hi = LSTM(hidden, name='es_lstm', recurrent_dropout=0.3, dropout=0.3)

       
# en
inp_en = Input(shape=(32, dims))
rnn_en_ = rnn_en(inp_en)
x = Dropout(0.3)(rnn_en_)
out_en = Dense(3, activation='softmax', name='en')(x)


# es
inp_hi = Input(shape=(32, dims))
rnn_hi_ = rnn_hi(inp_hi)
x = Dropout(0.3)(rnn_hi_)
out_hi = Dense(3, activation='softmax', name='es')(x)


cell_combined = GiretTwoCell(rnn_en.cell , rnn_hi.cell , hidden)

        
inp_enhi = Input(shape=(32, dims))
x = inp_enhi
x_att = x
x_att = Bidirectional(LSTM(32 , return_sequences=True, recurrent_dropout=0.3, dropout=0.3))( x )
bider_h = x_att 
x_att = Dropout(0.3)(x_att)
x_att = TimeDistributed(Dense(3, activation='softmax') )(x_att)
x_att = Lambda(lambda x : x[... , 1: ])(x_att)

x = Concatenate(-1)([x_att , x ])

x =  RNN(cell_combined, name='damn')(x)
# x = AttentionWithContext()(x)
out_enhi = Dense(3, activation='softmax', name='cm')(x)
        
model = Model( [inp_en , inp_hi , inp_enhi  ] , [ out_en , out_hi , out_enhi ] ) 

In [238]:
hidden = 100
numwords = weight_matrix.shape[0]
hidden_emd_dim = 300


embed = Embedding(numwords, hidden_emd_dim, weights=[weight_matrix], mask_zero=True)
# conv1 = Conv1D(64, 3, activation='relu', padding='valid',strides=1)
# pool1 = MaxPooling1D(2)
# conv2 = Conv1D(64, 5, activation='relu', padding='valid', strides=1)
# pool2 = MaxPooling1D(2)
# conv3 = Conv1D(128, 5, activation='relu', padding='valid', strides=1)
# pool3 = MaxPooling1D(2)
# conv3 = Conv1D(128, 5, activation='relu', padding='valid',strides=1)
# pool3 = MaxPooling1D(35)  # global max pooling


rnn_en = LSTM(hidden, name='en_lstm', recurrent_dropout=0.3, dropout=0.3)
rnn_hi = LSTM(hidden, name="hi_lstm", recurrent_dropout=0.3, dropout=0.3)
       
# en
inp_en = Input((None, ))
x = embed(inp_en)
# x = conv1(x)
# x = pool1(x)
# x = conv2(x)
# x = pool2(x)
# x = conv3(x)
# x = pool3(x)
x = Dropout(0.3)(x)
x = rnn_en(x)
out_en = Dense(3, activation='softmax')(x)


# es
inp_hi = Input((None, ))
x = embed(inp_hi)
# x = conv1(x)
# x = pool1(x)
# x = conv2(x)
# x = pool2(x)
# x = conv3(x)
# x = pool3(x)
x = Dropout(0.3)(x)
x = rnn_hi( x )
out_hi = Dense(3, activation='softmax')(x)


cell_combined = GiretTwoCell(rnn_hi.cell , rnn_en.cell , hidden)

inp_enhi = Input((None, ))
x = embed(inp_enhi)
# x = conv1(x)
# x = pool1(x)
# x = conv2(x)
# x = pool2(x)
# x = conv3(x)
# x = pool3(x)
x_att = x
x_att = Bidirectional(LSTM(32 , return_sequences=True, recurrent_dropout=0.3, dropout=0.3))( x )
bider_h = x_att 
x_att = Dropout(0.3)(x_att)
x_att = TimeDistributed(Dense(3, activation='softmax') )(x_att)
x_att = Lambda(lambda x : x[... , 1: ])(x_att)

x = Concatenate(-1)([x_att , x ])

x =  RNN(cell_combined, name='damn')(x)
# x = AttentionWithContext()(x)
out_enhi = Dense(3, activation='softmax', name='cm')(x)
        
model = Model( [inp_hi , inp_en , inp_enhi  ] , [ out_hi , out_en , out_enhi ] ) 

In [280]:
es = EarlyStopping(monitor='val_cm_f1', mode='max', verbose=1, patience=5, restore_best_weights=True)

In [281]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1])

In [282]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_66 (InputLayer)           (None, 32, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_22 (Bidirectional (None, 32, 64)       85248       input_66[0][0]                   
__________________________________________________________________________________________________
dropout_62 (Dropout)            (None, 32, 64)       0           bidirectional_22[0][0]           
__________________________________________________________________________________________________
time_distributed_18 (TimeDistri (None, 32, 3)        195         dropout_62[0][0]                 
__________________________________________________________________________________________________
input_64 (

In [283]:
en_x = np.concatenate([en_twitter_x, en_semeval_17_x])
en_y = np.concatenate([en_twitter_y, en_semeval_17_y])
es_x = np.concatenate([es_twitter_x, es_tass_x])
es_y = np.concatenate([es_twitter_y, es_tass_y])

In [284]:
# all_x = np.concatenate([en_semeval_17_x, en_twitter_x, es_tass_x, es_twitter_x, en_es_x_train])
# all_y = np.concatenate([en_semeval_17_y, en_twitter_y, es_tass_y, es_twitter_y, en_es_y_train])
all_x = np.concatenate([en_semeval_17_x, es_tass_x, es_twitter_x])
all_y = np.concatenate([en_semeval_17_y, es_tass_y, es_twitter_y])

In [285]:
import random
def train_generator(batch_size=4, lang="cm"):
    b = batch_size
    
    while True:
        
#         n3 = np.random.randint(0, en_es_x_train.shape[0] - batch_size, batch_size)


        if lang is "cm": 
            n1 = np.random.randint(0, en_x.shape[0] - batch_size, batch_size)
            n2 = np.random.randint(0, es_x.shape[0] - batch_size, batch_size)
            p = random.random()
            if p < 0.3:
                p = random.random()
                if p < 0.5:
                    n3 = np.random.randint(0, en_x.shape[0] - batch_size, batch_size)
                    x = [en_x[n1,:], es_x[n2,:], en_x[n3,:]]
                    y = [en_y[n1,:], es_y[n2,:], en_y[n3,:]]
                else:
                    n3 = np.random.randint(0, es_x.shape[0] - batch_size, batch_size)
                    x = [en_x[n1,:], es_x[n2,:], es_x[n3,:]]
                    y = [en_y[n1,:], es_y[n2,:], es_y[n3,:]]
            else:
                n3 = np.random.randint(0, en_es_x_train.shape[0] - batch_size, batch_size)
                x = [en_x[n1,:], es_x[n2,:], en_es_x_train[n3,:]]
                y = [en_y[n1,:], es_y[n2,:], en_es_y_train[n3,:]]
        elif lang is "en":
            n3 = np.random.randint(0, en_x.shape[0] - batch_size, batch_size)
            x = [en_x[n3,:], en_x[n3,:], en_x[n3,:]]
            y = [en_y[n3,:], en_y[n3,:], en_y[n3,:]]
        elif lang is "es":
            n3 = np.random.randint(0, es_x.shape[0] - batch_size, batch_size)
            x = [es_x[n3,:], es_x[n3,:], es_x[n3,:]]
            y = [es_y[n3,:], es_y[n3,:], es_y[n3,:]]
        elif lang is "unsup":
            n1 = np.random.randint(0, en_x.shape[0] - batch_size, batch_size)
            n2 = np.random.randint(0, es_x.shape[0] - batch_size, batch_size)
            n3 = np.random.randint(0, all_x.shape[0] - batch_size, batch_size)
            x = [en_x[n1,:], es_x[n2,:], all_x[n3,:]]
            y = [en_y[n1,:], es_y[n2,:], all_y[n3,:]]
            
                 
        
#         x = [en_x[n1,:], es_x[n2,:], all_x[n3,:]]
#         y = [en_y[n1,:], es_y[n2,:], all_y[n3,:]]
#         x = [ en_es_x_train[n3,:],  en_es_x_train[n3,:], en_es_x_train[n3,:]]
#         y = [en_es_y_train[n3,:], en_es_y_train[n3,:], en_es_y_train[n3,:]]
        
        yield x, y

In [286]:
gen = train_generator(32, lang="en")
model.fit_generator(
    generator=gen,
    steps_per_epoch=200,
    epochs=20,
    initial_epoch=0,
    validation_data=([en_es_x_train,en_es_x_train,en_es_x_train],[en_es_y_train,en_es_y_train,en_es_y_train]),
    callbacks=[es]
)

Epoch 1/20
200/200 [==============================] - 99s 494ms/step - loss: 2.9835 - en_loss: 1.0056 - es_loss: 1.0017 - cm_loss: 0.9762 - en_f1: 0.3275 - es_f1: 0.3266 - cm_f1: 0.3567 - val_loss: 3.2282 - val_en_loss: 1.0795 - val_es_loss: 1.0830 - val_cm_loss: 1.0657 - val_en_f1: 0.0321 - val_es_f1: 0.1118 - val_cm_f1: 0.1290
Epoch 2/20
200/200 [==============================] - 65s 327ms/step - loss: 2.7458 - en_loss: 0.9441 - es_loss: 0.9313 - cm_loss: 0.8704 - en_f1: 0.4483 - es_f1: 0.4659 - cm_f1: 0.5291 - val_loss: 3.1572 - val_en_loss: 1.0790 - val_es_loss: 1.0652 - val_cm_loss: 1.0130 - val_en_f1: 0.3614 - val_es_f1: 0.3692 - val_cm_f1: 0.2986
Epoch 3/20
200/200 [==============================] - 70s 349ms/step - loss: 2.6826 - en_loss: 0.9231 - es_loss: 0.9087 - cm_loss: 0.8508 - en_f1: 0.4839 - es_f1: 0.4845 - cm_f1: 0.5464 - val_loss: 3.1598 - val_en_loss: 1.0586 - val_es_loss: 1.0855 - val_cm_loss: 1.0158 - val_en_f1: 0.3208 - val_es_f1: 0.4056 - val_cm_f1: 0.3968
Epoch 4

In [287]:
model.evaluate([en_es_x_test,en_es_x_test,en_es_x_test],[en_es_y_test,en_es_y_test,en_es_y_test])

613/613 [==============================] - 2s 3ms/step


[3.199548551736804,
 1.0782622342980899,
 1.119319643515358,
 1.0019666784933692,
 0.4979645117849938,
 0.5129714421502529,
 0.5279189461012649]

In [288]:
gen = train_generator(32, lang="es")
model.fit_generator(
    generator=gen,
    steps_per_epoch=200,
    epochs=30,
    initial_epoch=20,
    validation_data=([en_es_x_train,en_es_x_train,en_es_x_train],[en_es_y_train,en_es_y_train,en_es_y_train]),
    callbacks=[es]
)

Epoch 21/30
200/200 [==============================] - 63s 316ms/step - loss: 2.7181 - en_loss: 0.9273 - es_loss: 0.9161 - cm_loss: 0.8746 - en_f1: 0.4807 - es_f1: 0.4805 - cm_f1: 0.5307 - val_loss: 2.9684 - val_en_loss: 0.9849 - val_es_loss: 1.0129 - val_cm_loss: 0.9706 - val_en_f1: 0.3642 - val_es_f1: 0.3680 - val_cm_f1: 0.4227
Epoch 24/30
200/200 [==============================] - 65s 325ms/step - loss: 2.6586 - en_loss: 0.9075 - es_loss: 0.9001 - cm_loss: 0.8510 - en_f1: 0.5004 - es_f1: 0.5064 - cm_f1: 0.5464 - val_loss: 3.0103 - val_en_loss: 1.0009 - val_es_loss: 1.0308 - val_cm_loss: 0.9785 - val_en_f1: 0.3912 - val_es_f1: 0.3855 - val_cm_f1: 0.4332
Epoch 25/30
200/200 [==============================] - 63s 314ms/step - loss: 2.5710 - en_loss: 0.8764 - es_loss: 0.8716 - cm_loss: 0.8231 - en_f1: 0.5302 - es_f1: 0.5338 - cm_f1: 0.5800 - val_loss: 2.9658 - val_en_loss: 1.0016 - val_es_loss: 1.0068 - val_cm_loss: 0.9574 - val_en_f1: 0.4110 - val_es_f1: 0.4077 - val_cm_f1: 0.4531
Epoc

In [289]:
model.evaluate([en_es_x_test,en_es_x_test,en_es_x_test],[en_es_y_test,en_es_y_test,en_es_y_test])

613/613 [==============================] - 2s 3ms/step


[2.8801771301235304,
 0.9730363803236558,
 0.9758537548790165,
 0.9312869851001907,
 0.43307197920455437,
 0.414544276856676,
 0.47206871213959634]

In [290]:
gen = train_generator(32, lang="unsup")
model.fit_generator(
    generator=gen,
    steps_per_epoch=200,
    epochs=50,
    initial_epoch=30,
    validation_data=([en_es_x_train,en_es_x_train,en_es_x_train],[en_es_y_train,en_es_y_train,en_es_y_train]),
    callbacks=[es]
)

Epoch 31/50
200/200 [==============================] - 64s 319ms/step - loss: 2.3080 - en_loss: 0.7980 - es_loss: 0.7802 - cm_loss: 0.7299 - en_f1: 0.6096 - es_f1: 0.6164 - cm_f1: 0.6543 - val_loss: 2.9139 - val_en_loss: 0.9517 - val_es_loss: 0.9950 - val_cm_loss: 0.9672 - val_en_f1: 0.4389 - val_es_f1: 0.4202 - val_cm_f1: 0.4625
Epoch 32/50
200/200 [==============================] - 66s 328ms/step - loss: 2.2438 - en_loss: 0.7618 - es_loss: 0.7629 - cm_loss: 0.7192 - en_f1: 0.6205 - es_f1: 0.6317 - cm_f1: 0.6543 - val_loss: 2.9691 - val_en_loss: 0.9527 - val_es_loss: 1.0303 - val_cm_loss: 0.9861 - val_en_f1: 0.4531 - val_es_f1: 0.3988 - val_cm_f1: 0.4570
Epoch 33/50
200/200 [==============================] - 66s 330ms/step - loss: 2.2081 - en_loss: 0.7464 - es_loss: 0.7489 - cm_loss: 0.7128 - en_f1: 0.6434 - es_f1: 0.6431 - cm_f1: 0.6643 - val_loss: 2.8913 - val_en_loss: 0.9556 - val_es_loss: 0.9767 - val_cm_loss: 0.9590 - val_en_f1: 0.4462 - val_es_f1: 0.4557 - val_cm_f1: 0.4862
Epoc

In [291]:
model.evaluate([en_es_x_test,en_es_x_test,en_es_x_test],[en_es_y_test,en_es_y_test,en_es_y_test])

613/613 [==============================] - 2s 4ms/step


[2.8389667408695813,
 0.9418664416131148,
 0.98001595840563,
 0.9170843568945009,
 0.5008024851416297,
 0.49733431868218675,
 0.5216923524271606]

In [295]:
gen = train_generator(32)
model.fit_generator(
    generator=gen,
    steps_per_epoch=200,
    epochs=100,
    initial_epoch=75,
    validation_data=([en_es_x_test,en_es_x_test,en_es_x_test],[en_es_y_test,en_es_y_test,en_es_y_test])
)

Epoch 76/100
200/200 [==============================] - 64s 320ms/step - loss: 1.4262 - en_loss: 0.6162 - es_loss: 0.4875 - cm_loss: 0.3225 - en_f1: 0.7276 - es_f1: 0.7991 - cm_f1: 0.8785 - val_loss: 3.1155 - val_en_loss: 0.9687 - val_es_loss: 1.0265 - val_cm_loss: 1.1203 - val_en_f1: 0.5433 - val_es_f1: 0.5181 - val_cm_f1: 0.6460
Epoch 77/100
200/200 [==============================] - 62s 312ms/step - loss: 1.3893 - en_loss: 0.6259 - es_loss: 0.4705 - cm_loss: 0.2929 - en_f1: 0.7206 - es_f1: 0.8048 - cm_f1: 0.8905 - val_loss: 3.1507 - val_en_loss: 0.9638 - val_es_loss: 1.0386 - val_cm_loss: 1.1484 - val_en_f1: 0.5417 - val_es_f1: 0.5461 - val_cm_f1: 0.6494
Epoch 78/100
200/200 [==============================] - 63s 313ms/step - loss: 1.3672 - en_loss: 0.6071 - es_loss: 0.4508 - cm_loss: 0.3093 - en_f1: 0.7327 - es_f1: 0.8184 - cm_f1: 0.8833 - val_loss: 3.2157 - val_en_loss: 0.9730 - val_es_loss: 1.0169 - val_cm_loss: 1.2259 - val_en_f1: 0.5265 - val_es_f1: 0.5474 - val_cm_f1: 0.6505
E

KeyboardInterrupt: 

In [294]:
model.evaluate([en_es_x_test,en_es_x_test,en_es_x_test],[en_es_y_test,en_es_y_test,en_es_y_test])

613/613 [==============================] - 2s 4ms/step


[3.1588352623032514,
 0.9760268991767485,
 1.0341843986005612,
 1.1486239554345317,
 0.5573675691789749,
 0.5354144665584284,
 0.6578083537142195]

# ! pip3 install keras-self-attention

# Hindi English Dataset

In [ ]:
with open("../data/IIITH_Codemixed.txt") as f:
    lines = f.readlines()[:-1]
print(len(lines))

In [ ]:
sents = {"0":-1 , "1" :1 , "2":0}
data = map( lambda x : x.strip().split("\t") , lines )
data = map( lambda x :{'sentiment': sents[x[3]] , 'text': x[1] } , data )
hi_en_data = list(data)

In [ ]:
X_train = [x['text'] for x in hi_en_data]
y_train = [x['sentiment'] for x in hi_en_data]

In [ ]:
from keras.utils import to_categorical

In [ ]:
y_train = to_categorical(y_train,num_classes=3)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
max_len = 32
zero_vector = [0 for _ in range(300)]
def get_x(data_):
    x_  = []
    for sent in data_:
        pred = list(multibpemb.embed(sent))
        if len(pred) >= 32:
            pred = pred[:32]
        else:
            counter = len(pred)
            while counter < max_len:
                pred.append(zero_vector)
                counter = counter + 1
        x_.append(pred)
    return np.array(x_)
X_train =  get_x(X_train)
X_test = get_x(X_test)